# Problem Definition (Business Understanding)
Spaceship Titanic is a fictional competition on Kaggle.

Objective: To predict whether new passengers will be teleported or not, based on the information that the passengers are "Transported" (the state of being teleported to another dimension).

This is a binary classification problem.
The model learns to predict the Transported value (True or False) of each passenger.


# About Dataset (Dataset Overview)
Data consists of two files, train.csv and test.csv:

File Description
train.csv Training data (Passenger information + Transported tag)
test.csv Passenger information only, data to be predicted

# Data Dictionary
- PassengerId: (Passenger ID (example: '0001_01') )
- HomePlanet: (The planet the passenger departed from (Earth, Europa, Mars))
- CryoSleep: (Was the passenger in sleep mode (True/False))
- Cabin: (Passenger's cabin (example: 'B/0/P'))
- Destination: (Destination planet (TRAPPIST-1e, etc.))
- Age: (Age)
- VIP: (VIP customer? (True/False))
- RoomService:(Room service spend (₡))
- FoodCourt: (Food spend)
- ShoppingMall: (Shopping spend)
- Spa: (Spa spend)
- VRDeck: (Virtual reality deck spend)
- Name: (Passenger's full name)
- Transported: (Destination (teleported?) → True/False (only in train.csv))

# Target Variable
Transported: Boolean

Type: Binary Classification

Target: True or False → This column will be predicted.